Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uTools.uconst as uconst
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature
import uniflocpy.uReservoir.IPR_simple_line as IPR_simple_line
import uniflocpy.uTools.plotly_workflow as plotly_workflow 
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uValidation.by_UniflocVBA as bvba

## Настройки расчета

In [ ]:
calc_options ={"step_lenth_in_calc_along_wellbore_m":25,
                "without_annulus_space":False,
                "solver_using":True}

## Данные по скважинам

Скв. 1Ф Исследование 4РМ

In [ ]:
pb_bar = 9 * 10**5
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293}

well_data = {"h_intake_mes_m" :1211,
"h_intake_vert_m" : 1211,
"h_bottomhole_mes_m" : 1757, 
"h_bottomhole_vert_m": 1757,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':155.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":240,
"fw_on_surface_perc":25}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.975, 8.495, 9.44, 10.365, 10.902, 11.272, 12.085, 12.907, 13.785, 14.67, 15.55],
                                 'h_mes_survey_m': [0, 957, 1057, 1157, 1211, 1257, 1357, 1457, 1557, 1657, 1757]})

Скв. 2Ф Исследование 5РМ

In [ ]:

pb_bar = 9 * 10**5
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293}

well_data = {"h_intake_mes_m" :1205.5,
"h_intake_vert_m" : 1205.5,
"h_bottomhole_mes_m" : 1605, 
"h_bottomhole_vert_m": 1605,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar': 114.35, 
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40,
"fw_on_surface_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.9, 1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435],
                                 'h_mes_survey_m': [0, 105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]})

Скв. 3Ф 6РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293,
'gamma_wat':1.04}

well_data = {"h_intake_mes_m" :1100,
"h_intake_vert_m" : 1100,
"h_bottomhole_mes_m" : 1775,
"h_bottomhole_vert_m": 1775,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':163.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":36/0.86,
"fw_on_surface_perc":23}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.1, 3.05, 4.95, 6.51, 8.23, 9.91, 12.02, 14.14, 16.35],
                                 'h_mes_survey_m': [0, 376, 576, 776, 976, 1176, 1376, 1576, 1775]})

Скв. 4Ф 7РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293,
'gamma_wat':1.064}

well_data = {"h_intake_mes_m" :1099,
"h_intake_vert_m" : 1099,
"h_bottomhole_mes_m" : 1663,
"h_bottomhole_vert_m": 1663,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":15,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':117.2,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40/0.86,
"fw_on_surface_perc":6.2}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.4, 1.28, 2.29, 3.82, 5.11, 6.75, 8.23, 10.0, 11.72],
                                 'h_mes_survey_m': [0, 263, 463, 663, 863, 1063, 1263, 1463, 1663]})

Скв. 5Ф 8РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293,
'gamma_wat':1.080}

well_data = {"h_intake_mes_m" :1079,
"h_intake_vert_m" : 1079,
"h_bottomhole_mes_m" : 1707,
"h_bottomhole_vert_m": 1707,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':150,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":154/0.86,
"fw_on_surface_perc":27}
real_measurements = pd.DataFrame({'p_survey_mpa': [1, 2.79, 4.3, 6.1, 7.8, 9.6, 11.3, 13.1, 15],
                                 'h_mes_survey_m': [0, 300, 500, 700, 900, 1100, 1300, 1500, 1707]})

Скв. 8Ф 12РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293,
'gamma_wat':1.064}

well_data = {"h_intake_mes_m" :1079,
"h_intake_vert_m" : 1079,
"h_bottomhole_mes_m" : 1711,
"h_bottomhole_vert_m": 1711,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":10,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':160.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":77.5/0.86,
"fw_on_surface_perc":20.6}
real_measurements = pd.DataFrame({'p_survey_mpa': [1, 9.95, 11.25, 12.15, 13.08, 14.11, 15.05, 16.05],
                                 'h_mes_survey_m': [0, 1078, 1210, 1310, 1410, 1510, 1610, 1711]})

Скв.2Ф Исследование 2Пл Памятно-Сасовское месторождение стр.93

In [ ]:
pb_bar = 16,7 * 10**5
mu_oil_cp = 10.05
fluid_data = {"rsb_m3m3" : 121.72,
"gamma_oil" : 0.83,
"gamma_gas" : 0.82}

well_data = {"h_intake_mes_m" :2519,
"h_intake_vert_m" : 2519,
"h_bottomhole_mes_m" : 2730, 
"h_bottomhole_vert_m": 2730,
             
"geothermal_grad_cm":0.027,
"t_wellhead_c":23.2,
"t_bottomhole_c" :76,
"t_earth_init_in_reservoir_c" : 76,
'p_bottomhole_bar':225,
"d_casing_inner_m" : 0.1553,
"d_tube_inner_m" : 0.076,
"qliq_on_surface_m3day":209.9,
"fw_on_surface_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [5.73, 8.12, 11.0, 14.15, 17.40, 19.13, 21.41, None],
                                  't_survey_c': [23.2, 32.0, 43.5, 55.0, 66.0, 70.2, 75.6, 76.0],
                                 'h_mes_survey_m': [0, 500, 1000, 1500, 2000, 2250, 2585, 2700]})

## Настройка модели скважины и расчет

In [ ]:
#well_data["t_wellhead_c"] = well_data["t_bottomhole_c"]


In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()

In [ ]:
blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1
blackoil_option.z_cor_number = 1
blackoil_option.pseudocritical_temperature_cor_number = 1
blackoil_option.pseudocritical_pressure_cor_number = 1
blackoil_option.rho_gas_cor_number = 1
blackoil_option.b_gas_cor_number = 1
blackoil_option.mu_dead_oil_cor_number = 2
blackoil_option.sigma_oil_gas_cor_number = 2
blackoil_option.sigma_wat_gas_cor_number = 1

In [ ]:

simple_well = self_flow_well.self_flow_well(fluid=1, reservoir = 0, pipe=0, temp_corr=1, **fluid_data,
                                            **well_data, **calc_options)

In [ ]:
simple_well.pipe.fluid_flow.fl.option = blackoil_option

In [ ]:
simple_well.pipe.hydr_cor.epsilon_friction_m = 0.0001

In [ ]:
simple_well.calc_all_from_down_to_up()

# Проверка с помощью UniflocVBA

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

In [ ]:
str_pvt = uniflocvba.PVT_encode_string(**fluid_data,tres_C =  well_data['t_earth_init_in_reservoir_c'])

In [ ]:
calc_along_coord = 0
flow_along_coord =0 
hydr_corr = 0
temp_method = 1
c_calibr_grav=1
c_calibr_fric=1
roughness_m=0.0001
q_gas_sm3day=0
out_curves_num_points=int(well_data["h_bottomhole_mes_m"]/calc_options['step_lenth_in_calc_along_wellbore_m'])

In [ ]:
result =uniflocvba.MF_p_pipeline_atma(
    well_data["qliq_on_surface_m3day"],
    well_data["fw_on_surface_perc"],
    [
        [0,0],
         [well_data["h_bottomhole_mes_m"], well_data["h_bottomhole_mes_m"]
         ]
    ],
    uconst.bar2atm(well_data["p_bottomhole_bar"]),
    well_data["t_bottomhole_c"],
    calc_along_coord,
    flow_along_coord,
    str_pvt,
    [[0,well_data["d_tube_inner_m"] * 1000],
     [well_data["h_intake_mes_m"],well_data["d_casing_inner_m"] * 1000],
     [well_data["h_bottomhole_mes_m"],well_data["d_casing_inner_m"] * 1000]],
    hydr_corr,
    [[0,well_data["t_wellhead_c"]], [well_data["h_bottomhole_mes_m"],well_data["t_bottomhole_c"]]],
    temp_method,
    c_calibr_grav,
    c_calibr_fric,
    roughness_m,
    q_gas_sm3day,
    out_curves_num_points)

In [ ]:
result_vba = bvba.covert_result_from_vba_to_df(result)

## Выгрузка расчитанных значений и объединение с фактическими данными

In [ ]:
result_df = simple_well.data.get_data_as_df()
result_df.to_excel('test.xlsx')
result_df = result_df.set_index('well.h_calculated_mes_m')

Добавление результатов фактических исследований

In [ ]:
real_measurements["p_survey_bar"] = real_measurements["p_survey_mpa"] * 10
real_measurements = real_measurements.set_index(real_measurements['h_mes_survey_m'])
real_measurements.index.name = 'well.h_calculated_mes_m'

result_df = result_df.join(real_measurements, how = 'outer')

Добавление результатов расчета через VBA

In [ ]:
result_df = result_df.join(result_vba, how = 'outer')

In [ ]:
(real_measurements["p_survey_bar"][0] - simple_well.p_wellhead_bar) / real_measurements["p_survey_bar"][0] * 100

In [ ]:
(real_measurements["p_survey_bar"][0] - result_vba['p_calculated_bar_vba'][0]) / real_measurements["p_survey_bar"][0] * 100

In [ ]:
(result_vba["p_calculated_bar_vba"][0] - simple_well.p_wellhead_bar) / result_vba["p_calculated_bar_vba"][0] * 100

## Построение графиков

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey'], ['p_calculated_bar_vba'], ['t_calculated_c_vba']]
antipatterns = ['mu', 'h_mes_survey', 'mpa']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

## Встроенный блок многовариативного анализа

In [ ]:
results = []
#for i in range(10, int(well_data['qliq_on_surface_m3day'] * 2), int((well_data['qliq_on_surface_m3day'] * 2 - 10) / 7) ):
for i in range(1, 190, 15):

    print(i)
    simple_well.data.clear_data()
    simple_well.qliq_on_surface_m3day = i
    simple_well.calc_all_from_down_to_up()
    this_result_df = simple_well.data.get_data_as_df()
    this_result_df = this_result_df.set_index('well.h_calculated_mes_m')
    results.append(this_result_df)
    
    this_result_df = this_result_df.add_prefix(f"qliq_m3day: {i}, ")
    try:
        result_df_to_plot = result_df_to_plot.join(this_result_df)
    except:
        result_df_to_plot = this_result_df.copy()

In [ ]:
result_df_analysis = None
for i in results:
    qliq_on_surface_m3day = i['well.qliq_on_surface_m3day'][0]
    this_df = pd.DataFrame({'well.p_bottomhole_bar': i['well.p_bottomhole_bar'][0],
                           'ipr.p_reservoir_bar': i['ipr.p_reservoir_bar'][0],
                           'well.p_calculated_bar': i['well.p_calculated_bar'][0],}, index = [qliq_on_surface_m3day])
    try:
        result_df_analysis = result_df_analysis.append(this_df)
    except:
        result_df_analysis = this_df.copy()

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_analysis, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', iplot_option = True)


In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey']]
antipatterns = ['mu', 'h_mes_survey', 'mpa']
plotly_workflow.plot_by_patterns(results[0], group_patterns, antipatterns, plot_name='Распределение свойств по глубине')

Результаты в одном DataFrame

In [ ]:
result_df = result_df_to_plot

Результаты по одной скважине

In [ ]:
result_df = results[0]

## Продолжение построения распределения свойств по глубине для выбранной скважины

Почему не сходится

коэффициент сверхжимаемости - DAK точнее, ошибка в псевдокритических значениях - в общем причина не в этом

попутаны номера режимов, хотя карта нормальная

массовый расход не сходится

q_gas

### Название всех сохраненных параметров для выбора или не выбора их при построении графиков

In [ ]:
for i in result_df.columns:
    print(i)

In [ ]:
group_patterns = ["flow_regime"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["Ek"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["p_calculated_bar"]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["p_wellhead"]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["rho"]
antipatterns = ['cor_number', 'earth']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [['roughness']]
antipatterns = ['cor_number', 'earth', 'with','cor']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [['eps'],["c_Rough"]]
antipatterns = ['cor_number', 'earth', 'with','cor']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["gas_fraction"],['c_gasfrac'],['liquid_content']]
antipatterns = ['cor_number', 'earth', 'with','cor']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["mu"]
antipatterns = ['earth', 'cor_number']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["_m3day"]]
antipatterns = ['earth', 'on_surface']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["perc",'fw']
antipatterns = ['earth']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["d_m"],['diam']]
antipatterns = ['earth', 'mes_m']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["rs","m3m3"]
antipatterns = ['earth']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["vsl_msec"]
antipatterns = ['secm', 'msecpam', 'earth', 't_calculated', 'mass_flowrate', 'gasfrac']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["vm"]
antipatterns = ['secm', 'msecpam', 'earth', 't_calculated', 'mass_flowrate', 'gasfrac']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["val_number_Fr"]
antipatterns = ['secm', 'msecpam', 'earth', 't_calculated', 'mass_flowrate', 'gasfrac']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["rho_liq"]
antipatterns = ['secm', 'msecpam', 'earth', 't_calculated', 'mass_flowrate', 'gasfrac']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["c_vl"],['c_vg'],['vsl']]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["pb"]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [ 'density']
antipatterns = ['tpb', 'number', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["number_Re"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["liquid_content_with_Pains"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["angle_grad"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["Nlv"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["grad"]
antipatterns = ['tpb', 'well_profile','percent','angle','therm','t_c']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["angle_correction"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["mass_flowrate"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ["number",'calculation']
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = ['time_calculated_sec']
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
trace1 = go.Scatter3d(
    x=simple_well.well_profile.x_displacement_m,
    y=simple_well.well_profile.x_displacement_m * 0,
    z=simple_well.well_profile.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m),
    y=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m) * 0,
    z=simple_well.well_profile.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')